# First Assignment of AI

Professors: Fadaei and Yaghoob Zade

Mohamad Mahdi Samadi

810101465

In [1]:
import pandas as pd
import random
import numpy as np


Defined numbers used in code:

At first we have some parameters given in the problem description.

In [2]:

file_path = 'snacks.csv'
min_value = 20
max_weight = 15
min_snack = 3
max_snack = 5
p_size = 200

The fitness function applies a number between zero and ten to each chromosome which is distributed among parameters i defined. First three parameters are obvious. But the last one is defined as value/weight for each snack which is very imortant to us because there is a limit for total space and value of snacks.

In [3]:
value_rate = 2
weight_rate = 2
diversity_rate = 2
density_rate = 4
p_crossover = 0.5
p_mutation = 0.1

There are infinite numbers between 0 and 100. We could break the interval to some sub-intervals and calculations whould be much easier.

In [4]:
interval_cnt = 5
interval_len = 100//interval_cnt

This is start of the program.
At first given csv file is loaded. It contains information about snacks such as their names, values and weights.

In [5]:
df = pd.read_csv(file_path)
weight = [int(w)/(interval_cnt) for w in list(df['Available Weight'])]
value = [int(v)/(interval_cnt) for v in list(df['Value'])]
name = ([n for n in list(df['Snack'])])
density = [value[i]/weight[i] for i in range(len(value))]
snacks_num = len(value)

After loading data we have to create an initial generation for start. Let's define some concepts used in this algorithm.

Generation: formed by some chromosomes.

Chromosome: Includes 19 genes in this particular case. It represents a knapsack with its snacks

Gene: Each gene represent a snack in this problem. It has an integer value between zero and number of intervals. For example if a gene has value 7 it means that we pack 70 percent of that snack into our knapsack. So the gained value and used space by this snack whould be 70 percent of what is written in the file.

Here is a function to generate first population. We can specify the number of snacks of each knapsack in two ways. It could be a random number between 0 to number of all snacks or it could be between lower and upper bound of number of snacks defined in the problem. The first method is more logical because we want the first generation to be completely random but the second one leads us to an complete answer in much less time. Here we use the second one so that running time of the program whould be shorter and easier for TA to check the final answer. After that we have to pick random snacks with random value. We know that there is no guarantee there would be a complete answer between them. Actually it's unlikely to happen. 

In [6]:
def generate_population():
    population = []
    for _ in range(p_size):
        chromosome = [0] * snacks_num
        snacks_cnt = random.randint(min_snack, max_snack)
        #snacks_cnt = random.randint(0, 19)
        chosen_snacks = 0
        while chosen_snacks < snacks_cnt:
            ind = random.randint(0, snacks_num-1)
            if (chromosome[ind] == 0):
                chromosome[ind] = random.randint(1, interval_cnt)
                chosen_snacks += 1
        population.append(chromosome)
    return population

This function decodes chromosomes and will be used in the next steps.

In [7]:
def decode_chromosome(chromosome):
    total_weight, total_value, total_snacks, total_density = 0, 0, 0, 0    
    for i in range(len(chromosome)):
        gene = chromosome[i]
        if gene != 0:
            total_weight += gene * weight[i]
            total_value += gene * value[i]
            total_snacks += 1
            total_density += density[i]
    return [total_weight, total_value, total_snacks, total_density/len(chromosome)]


These are the functions used to calculate fitness of each chromosome. There are 4 factors considered.

Value: if it's equal to or more than minimum asked value, It gets the whole score of this part. If not and it gets a part of the score (linear relationship)

Weight: It's exatly like value.

Density: It's defined as mean of density for all the genes in chromosome.

All of them together makes the fitness out of 10.

In [8]:
def rate_value(v):
    return value_rate * min(1, v/min_value)

def rate_weight(w):
    if w > 0:
        return weight_rate * min(1, max_weight/w)
    return 0

def rate_diversity(d):
    if d >= min_snack and d <= max_snack:
        return diversity_rate
    return 0

def rate_density(d):
    return min(d, density_rate)

def calc_fitness(chromosome):
    fitness = 0
    total_weight, total_value, total_snacks, total_density = decode_chromosome(chromosome)
    
    fitness += rate_weight(total_weight)
    fitness += rate_value(total_value)
    fitness += rate_diversity(total_snacks)
    fitness += rate_density(total_density)
    return fitness


Now we can take a sample of them. The higher the fitness is, The more the chance of that chromosome to appear in next generation. I used fitness proportionate selection (FPS) for this part. It means that the chance of getting picked has linear relationship with fitness.

In [9]:
def calc_probabilities(fitness):
    total_fitness = np.sum(fitness)
    probabilities = [f/total_fitness for f in fitness]
    return probabilities

def take_sample(population, probabilities):
    sample_population = random.choices(population, weights=probabilities, k=len(population))
    return sample_population
def shuffle_sample(sample_generation):
    for _ in range(7):
        random.shuffle(sample_generation)


This is the crossover function. I used 1 point method. It's easy and fast but not the best way to improve our chromosomes. 

In [10]:
def crossover(population):
    i = 0
    while i < len(population) - 1:
        cross_point = int(snacks_num*0.5)
        first_child = population[i][:cross_point] + population[i+1][cross_point:]
        second_child = population[i][cross_point:] + population[i+1][:cross_point]
        population[i], population[i+1] = first_child, second_child
        i += 2


Here is the main reasone why our chromosomes improves after each generation. After crossovering pair of chromosomes we can mutate them too. It means that each gene with probability of p_mutation whould change and we hope it leads us to a better chromosome. p_mutation is different for each gene. If a chromosome is already good then we don't have to change it and p whould be small and if it's not good we will change it.

In [11]:
def mutate(population, fitness):
    for i in range(len(population)):
        flip_prob = 1 - fitness[i]/10
        for j in range(len(population[i])):
            if random.randint(0, 1000)/1000 < flip_prob:
                population[i][j] = 1 - population[i][j]


These two functions are used to find out if an answer is complete and then print it.

In [12]:
def is_answer(phenotype):
    total_weight, total_value, total_snacks = phenotype
    if (total_snacks >= min_snack and total_snacks <= max_snack):
        if (total_weight <= max_weight and total_value >= min_value):
            return True
    return False

def show_answer(chromosome, phenotype):
    print(chromosome)
    print('total weight:', phenotype[0])
    print('total value:', phenotype[1])
    print('snacks num:', phenotype[2])
    print()
    for i in range(len(chromosome)):
        gene = chromosome[i]
        if (gene != 0):
            print(name[i])
            print('value', value[i] * gene, 'weight', weight[i] * gene)


We've prepared everything is needed to run the algorithm. Now let's put them together in the main function.

In [13]:
def main():
    population = generate_population()
    i = 1
    while True:
        i += 1
        fitness = [calc_fitness(chromosome) for chromosome in population]
        probabilities = calc_probabilities(fitness)
        sample_population = take_sample(population, probabilities)
        shuffle_sample(sample_population)
        crossover(sample_population)
        mutate(sample_population, fitness)
        for chromosome in sample_population:
            phenotype = decode_chromosome(chromosome)
            not_negative = True
            for gene in chromosome:
                if gene < 0:
                    not_negative = False
            if (is_answer(phenotype[:3]) and not_negative):
                show_answer(chromosome, phenotype)
                print('a complete answer has been found after', i, 'generations')
                return
    
    
main()

[0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
total weight: 11.8
total value: 20.0
snacks num: 4

Nani
value 1.0 weight 1.0
Jooj
value 15.0 weight 7.0
Tordilla
value 1.8 weight 2.0
Saghe-Talaei
value 2.2 weight 1.8
a complete answer has been found after 238 generations


There are some qustions at the end of assignment description.

1. with small initial population we have a harder way to reach a complete solution. But it's fast because of the small space we take and faster to procces them. On the other hand if we start with big population there are more kinds of chromosomes and better combinations. But it's slow and takes more time and space to run. SO it's important to start with a reasonable size of initial population.

2. It causes a slower algorithm than current one because of the bigger population. It also might increase our chance of finding a complete answer in fewer generations if the new chromosomes are strong enough.

3. No we can't use just one of them because they have different purposes. Crossover combines two different chromosomes and make an better one but mutation is performed on single chromosome and improves it. 

4. I tested these and they all worked
    1. generate more accurate initial population (it's not logical because the point of GA is to start with complete randomness)
    2. generate bigger initial population (about 200)
    3. use a more accurate rating system.
    4. In the crossover step we could sort them by fitness and then pair them. In this case stronger chromosomes come together.



5. The stop in the evolution of chromosomes might happen in these cases:
    1. our initial population wasn't diverse enough
    2. mutation and crossover doesn't happen at the right time. It's important to have a logical reason for the used p_crossover and p_mutation so that they help us find an answer faster.



6. is there a complete solution for the problem or not? if not why are we searching for it? There is a greedy algorithm for the fractional knapsack problem. First sort the snacks by their density (value/weight). It's only logical to choose the snacks from the one with higher density. After that if we haven't reached the min_snack number we can choose an epsilon of some other snacks and remove that weight from current snacks. But if we pass the max_snack number or we can't fill the knapsack by given snacks there is no complete answer for this problem. But what if the problem has complete answer(s) and we can't reach them? maybe the initial generation or crossover or mutation isn't good enough. We can set a max limit for the generation and stop when reaching it.